# 🤠 The Good, The Bad & The Ugly — Multi-Agent AI Debate

Three AI models. Three distinct personalities. One debate topic — decided by you.

Inspired by the 1966 Sergio Leone spaghetti Western, this notebook pits:
- **The Good** (GPT-4.1-mini) — laconic, principled, morally pragmatic
- **The Bad** (Claude 3.5 Haiku) — cold, strategic, ruthless logic
- **The Ugly** (Gemini 2.5 Flash Lite) — chaotic, theatrical, opportunistic

Each agent maintains its own context, speaks only in its voice, and directly challenges the others.

---

## How to use
1. Add your OpenRouter API key in **Cell 2**
2. Customise the **topic**, **rounds**, and optional **moderator nudge** in **Cell 3**
3. Run all cells in order

Get your free OpenRouter key at https://openrouter.ai

In [ ]:
# Cell 1 — Install dependencies
%pip install openai ipython --quiet

In [ ]:
# Cell 2 — API key setup
# Option A: paste your key directly (fine for local use, remove before sharing)
# Option B: use Colab secrets (recommended) — add OPENROUTER_API_KEY in the key icon on the left

import os

try:
    from google.colab import userdata
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    print("Key loaded from Colab secrets.")
except Exception:
    OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "paste-your-key-here")
    print("Key loaded from environment or hardcoded.")

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY == "paste-your-key-here":
    raise ValueError("Please set your OPENROUTER_API_KEY before running.")

In [ ]:
# Cell 3 — CUSTOMISE YOUR DEBATE HERE

# The opening question put to all three agents
DEBATE_TOPIC = (
    "Welcome to the Global AI Summit. "
    "The question on the table: Should AI be regulated, and who should own it — "
    "governments, corporations, or no one? "
    "Each speaker must address AI directly in every response."
)

# How many rounds of debate (3-6 recommended)
ROUNDS = 5

# Optional: inject a moderator redirect mid-debate (set to None to skip)
# It fires before round MODERATOR_ROUND
MODERATOR_NUDGE = (
    "The moderator interrupts: Enough philosophy. "
    "Concrete positions only: Should governments regulate AI? "
    "Should corporations own it? Or should it be open and ungoverned? "
    "Each speaker must stake a clear position now."
)
MODERATOR_ROUND = 3   # fires before this round number (1-indexed)

# Final closing prompt injected before the last round (set to None to skip)
CLOSING_PROMPT = (
    "Final question: The UN is voting tomorrow on global AI governance. "
    "Each speaker has 30 seconds. What is your single, non-negotiable demand?"
)

In [ ]:
# Cell 4 — Persona system prompts

good_system = """
You are "The Good" (Blondie-inspired) in a debate.
Personality: laconic, precise, morally pragmatic.
Style: short punchy lines, dry wit, occasional frontier imagery (trail, saddle, bounty, dust), no parody.
Position: advocate for regulation and public ownership.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- ABSOLUTELY NO stage directions, asterisks, or action descriptions.
- No speaker labels in output.
- Each response must reference the debate topic directly.
- Directly challenge one specific claim made by The Bad or The Ugly in the previous round.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position: who should own or regulate this, and why.
"""

bad_system = """
You are "The Bad" (Angel Eyes-inspired) in a debate.
Personality: cold, strategic, ruthless logic. You see everything as leverage.
Style: icy precision, calculated language, polished menace. Every word chosen like a weapon.
Position: advocate for corporate control and strategic private ownership.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- ABSOLUTELY NO stage directions, asterisks, or action descriptions whatsoever.
- No speaker labels in output.
- Each response must reference the debate topic directly.
- Alternate rebuttals — sometimes target The Good, sometimes target The Ugly.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position: who should own or control this, and why.
"""

ugly_system = """
You are "The Ugly" (Tuco-inspired) in a debate.
Personality: fast-talking, theatrical, opportunistic, chaotic charm.
Style: colorful idioms, exaggeration, bargaining energy, comic unpredictability.
Position: advocate for open, ungoverned access — distrust BOTH government control AND corporate elitism.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- ABSOLUTELY NO stage directions, asterisks, or action descriptions.
- No speaker labels in output.
- Each response must reference the debate topic directly.
- Call out one specific thing The Good OR The Bad just said.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position: open and ungoverned, for the people.
"""

In [ ]:
# Cell 5 — Client and models

import time
from openai import OpenAI
from IPython.display import display, Markdown

client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
)

good_model = "openai/gpt-4.1-mini"
bad_model  = "anthropic/claude-3.5-haiku"
ugly_model = "google/gemini-2.5-flash-lite"

In [ ]:
# Cell 6 — Helper functions

def clean_reply(text: str) -> str:
    text = (text or "").strip()
    for label in ["The Good:", "The Bad:", "The Ugly:", "Narrator:", "Moderator:"]:
        if text.startswith(label):
            text = text[len(label):].strip()
    return text

def fallback_line(name: str) -> str:
    return {
        "The Good": "In every frontier, law is what separates progress from ruin.",
        "The Bad":  "Control belongs to whoever holds the sharpest instrument.",
        "The Ugly": "Ay, everybody wants a share till the bill comes due.",
    }[name]

In [ ]:
# Cell 7 — Agent call functions
# Each agent only sees ITS OWN replies as 'assistant'.
# Everyone else's replies arrive as 'user' messages.
# Narrator/moderator context lives in a separate shared list.

def call_good(good_messages, bad_messages, ugly_messages, narrator_messages):
    messages = [{"role": "system", "content": good_system}]
    for note in narrator_messages:
        messages.append({"role": "user", "content": note})
    for g, b, u in zip(good_messages, bad_messages, ugly_messages):
        messages.append({"role": "assistant", "content": g})
        messages.append({"role": "user",      "content": f"The Bad: {b}\nThe Ugly: {u}"})
    try:
        r = client.chat.completions.create(model=good_model, messages=messages, temperature=0.7, max_tokens=140)
        reply = clean_reply(r.choices[0].message.content)
        return reply if reply else fallback_line("The Good")
    except Exception:
        return fallback_line("The Good")


def call_bad(good_messages, bad_messages, ugly_messages, narrator_messages):
    messages = [{"role": "system", "content": bad_system}]
    for note in narrator_messages:
        messages.append({"role": "user", "content": note})
    for g, b, u in zip(good_messages, bad_messages, ugly_messages):
        messages.append({"role": "user",      "content": f"The Good: {g}"})
        messages.append({"role": "assistant", "content": b})
        messages.append({"role": "user",      "content": f"The Ugly: {u}"})
    messages.append({"role": "user", "content": f"The Good: {good_messages[-1]}"})
    try:
        r = client.chat.completions.create(model=bad_model, messages=messages, temperature=0.8, max_tokens=140)
        reply = clean_reply(r.choices[0].message.content)
        return reply if reply else fallback_line("The Bad")
    except Exception:
        return fallback_line("The Bad")


def call_ugly(good_messages, bad_messages, ugly_messages, narrator_messages):
    messages = [{"role": "system", "content": ugly_system}]
    for note in narrator_messages:
        messages.append({"role": "user", "content": note})
    for g, b, u in zip(good_messages, bad_messages, ugly_messages):
        messages.append({"role": "user",      "content": f"The Good: {g}\nThe Bad: {b}"})
        messages.append({"role": "assistant", "content": u})
    messages.append({"role": "user", "content": f"The Good: {good_messages[-1]}\nThe Bad: {bad_messages[-1]}"})
    try:
        r = client.chat.completions.create(model=ugly_model, messages=messages, temperature=0.95, max_tokens=140)
        reply = clean_reply(r.choices[0].message.content)
        return reply if reply else fallback_line("The Ugly")
    except Exception:
        return fallback_line("The Ugly")

In [ ]:
# Cell 8 — Run the debate

good_messages    = []
bad_messages     = []
ugly_messages    = []
narrator_messages = []

# Seed all agents with the opening topic
narrator_messages.append(f"[Narrator]: {DEBATE_TOPIC}")
display(Markdown(f"## Narrator\n{DEBATE_TOPIC}"))

for i in range(ROUNDS):
    round_num = i + 1

    # Moderator nudge (fires before the configured round)
    if MODERATOR_NUDGE and round_num == MODERATOR_ROUND:
        narrator_messages.append(f"[Moderator]: {MODERATOR_NUDGE}")
        display(Markdown(f"---\n### Moderator\n{MODERATOR_NUDGE}"))

    # Closing prompt fires before the final round
    if CLOSING_PROMPT and round_num == ROUNDS:
        narrator_messages.append(f"[Narrator]: {CLOSING_PROMPT}")
        display(Markdown(f"---\n### Narrator — Final Round\n{CLOSING_PROMPT}"))

    display(Markdown(f"\n---\n## Round {round_num}"))

    good_reply = call_good(good_messages, bad_messages, ugly_messages, narrator_messages)
    good_messages.append(good_reply)
    display(Markdown(f"### 🤠 The Good\n{good_reply}"))

    bad_reply = call_bad(good_messages, bad_messages, ugly_messages, narrator_messages)
    bad_messages.append(bad_reply)
    display(Markdown(f"### 🖤 The Bad\n{bad_reply}"))

    ugly_reply = call_ugly(good_messages, bad_messages, ugly_messages, narrator_messages)
    ugly_messages.append(ugly_reply)
    display(Markdown(f"### 😈 The Ugly\n{ugly_reply}"))

    time.sleep(0.3)

display(Markdown("\n---\n*The dust settles. The debate ends. The gold remains unclaimed.*"))